<a href="https://colab.research.google.com/github/robitussin/CCADMACL_EXERCISES/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [238]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

In [239]:
housing = fetch_california_housing(as_frame=True)

df_house_features = pd.DataFrame(housing.data, columns=housing.feature_names)
df_house_target = pd.DataFrame(housing, columns=['target'])

#concat
df = pd.concat([df_house_features, df_house_target], axis=1)

In [240]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


## Base Model

In [241]:
score_list = {}

In [242]:
X1 = df.drop(['target'],axis=1)
y1 = df['target']

X_train,X_test,y_train,y_test = train_test_split(X1,y1,test_size=0.5,random_state=51)

In [243]:
mlr_model = LinearRegression()

mlr_model.fit(X_train, y_train)
y_pred = mlr_model.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
score_list["Base Model RMSE (MLR)"] = rmse

print(f"Root Mean Squared Error is {rmse}")

Root Mean Squared Error is 0.7251207834475498


1. Use any filter method to select the best features

### Mutual Info Regression

In [244]:
from sklearn.feature_selection import mutual_info_regression
threshold = 5  # the number of most relevant features
high_score_features = []
feature_scores = mutual_info_regression(housing.data, housing.target.to_numpy(), random_state=0)

In [245]:
for score, f_name in sorted(zip(feature_scores, housing.data.columns), reverse=True)[:threshold]:
        print(f_name, score)
        high_score_features.append(f_name)
df_house_norm_mic = df[high_score_features]

Longitude 0.4018195828289901
MedInc 0.38764599736258987
Latitude 0.3706566756286751
AveRooms 0.10362300978707584
AveOccup 0.07254907984228254


In [246]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [247]:
X2 = df.drop(['target', 'HouseAge', 'Population', 'AveRooms'],axis=1)
y2 = df['target']

X_train,X_test,y_train,y_test = train_test_split(X2,y2,test_size=0.5,random_state=51)

In [248]:
mlr_model = LinearRegression()

mlr_model.fit(X_train, y_train)
y_pred = mlr_model.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
score_list["Mutual Info Reg. RMSE (MLR)"] = rmse

print(f"Root Mean Squared Error is {rmse}")

Root Mean Squared Error is 0.7384701230012191


2. Use any wrapper method to select the best features

In [249]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

threshold = 5 # the number of most relevant features
model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
selector = RFE(model_rf, n_features_to_select=5, step=1)

selector = selector.fit(housing.data, housing.target.values.ravel())
selector_ind = selector.get_support()
df_house_rfe = df_house_features.iloc[:, selector_ind]
df_house_rfe.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude'], dtype='object')

In [250]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [251]:
X3 = df.drop(['target', 'AveBedrms', 'Population', 'Longitude'],axis=1)
y3 = df['target']

X_train,X_test,y_train,y_test = train_test_split(X3,y3,test_size=0.5,random_state=51)

In [252]:
mlr_model = LinearRegression()

mlr_model.fit(X_train, y_train)
y_pred = mlr_model.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
score_list["Wrapper Method RMSE (MLR)"] = rmse

print(f"Root Mean Squared Error is {rmse}")

Root Mean Squared Error is 0.8008054191177062


3. Use any embedded method to select the best features

In [253]:
from sklearn.feature_selection import SelectFromModel

X4 = df.drop(['target'],axis=1)
y4 = df['target']

X_train,X_test,y_train,y_test = train_test_split(X4,y4,test_size=0.5,random_state=42)

model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_rf.fit(X_train, y_train)

sel_sfm = SelectFromModel(model_rf, prefit=True)
sel_sfm_index = sel_sfm.get_support()
df_house_sfm = df_house_features.iloc[:, sel_sfm_index]
df_house_sfm.columns

Index(['MedInc'], dtype='object')

In [254]:
y_pred = model_rf.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
score_list["Embedded RMSE (RF)"] = rmse
print(rmse)

0.7689261239415338


### Root Mean Square Errors

In [255]:
score_list = list(score_list.items())

for alg,score in score_list:
    print(f"{alg} Score is {str(score)[:6]} ")

Base Model RMSE (MLR) Score is 0.7251 
Mutual Info Reg. RMSE (MLR) Score is 0.7384 
Wrapper Method RMSE (MLR) Score is 0.8008 
Embedded RMSE (RF) Score is 0.7689 
